In [1]:
!pip install diffusers transformers accelerate sagemaker --upgrade --quiet

In [2]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
assert sess.boto_region_name in ["us-east-2", "us-east-1"] , "region must be us-east-2 or us-west-2, due to instance availability"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::618535004667:role/service-role/AmazonSageMaker-ExecutionRole-20240305T163986
sagemaker bucket: sagemaker-us-east-1-618535004667
sagemaker session region: us-east-1


In [3]:
# create s3 uri
s3_model_path = f"s3://{sess.default_bucket()}/sdxl_turbo2/model.tar.gz"

In [5]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data=s3_model_path,        # path to your model.tar.gz on s3
    transformers_version='4.37.0',
    pytorch_version='2.1.0',
    py_version='py310',
    role=role,
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type="ml.g4dn.xlarge",
    #volume_size = 120 # volumen size GB
)

-----------!

In [6]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64

# helper decoder
def decode_base64_image(image_string):
    base64_image = base64.b64decode(image_string)
    buffer = BytesIO(base64_image)
    return Image.open(buffer)

# display PIL images as grid
def display_image(image=None,width=500,height=500):
    img = image.resize((width, height))
    display(img)

In [11]:
prompt = "Una mansión con vista al mar, con dos niños y un perro jugando en el patio de esta."

# run prediction
response = predictor.predict(data={
  "inputs": prompt,
  "parameters": {
    "num_inference_steps": 1,
    "num_images_per_prompt": 1,
    "guidance_scale":0.0,
    }
  }
)

# decode and display image
response["generated_images"][0]

'...'